In [1]:
# Cell 1 – Install dependencies
%pip -q install requests beautifulsoup4 lxml pandas tqdm requests-cache pyarrow


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Cell 2 – Imports & config
from __future__ import annotations

import re
import json
import time
import random
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone
from pathlib import Path
from typing import Dict, Iterable, List, Optional

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

try:
    import requests_cache
except ModuleNotFoundError:
    requests_cache = None

# ── Output directory ────────────────────────────────────────────────────────
OUTPUT_DIR = Path.cwd() / "assets"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ── HTTP cache (speeds up reruns; set to False to disable) ──────────────────
CACHE_PATH           = OUTPUT_DIR / "http_cache"
USE_HTTP_CACHE       = True
CACHE_EXPIRE_SECONDS = 60 * 60 * 24   # 24 h

# ── Polite crawl settings ───────────────────────────────────────────────────
REQUEST_DELAY_RANGE = (0.15, 0.45)   # seconds between requests
MAX_WORKERS         = 6              # concurrent HamroPatro profile fetches

# ── ID discovery: HamroPatro range scan ────────────────────────────────────
# IDs observed in your data start at ~339,300. With ~3,500 candidates the
# range tops out around ~343,000. We use a tight window + threading so the
# scan finishes in ~5–10 minutes instead of 1.5 hours.
HAMRO_ID_RANGE_START            = 339_000   # just below first known ID
HAMRO_ID_RANGE_END              = 343_500   # generous upper bound (~4,500 IDs total)
RANGE_SCAN_WORKERS              = 12        # threads for the range scan (safe for this site)

# ── Minimum expected candidates (WARN only, never crash) ───────────────────
# Original notebook had raise RuntimeError if < 3300 → that blocked everything.
# Now we just print a warning.
MIN_EXPECTED_CANDIDATES = 3300

# ── Feature flags ──────────────────────────────────────────────────────────
FALLBACK_FETCH_2082LIVE_PROFILE = True  # fetch 2082.live profile for missing edu

# ── Base URLs ──────────────────────────────────────────────────────────────
BASE_2082LIVE = "https://2082.live"
BASE_HAMRO    = "https://election.hamropatro.com"

# ── Output files ───────────────────────────────────────────────────────────
INDEX_JSONL    = OUTPUT_DIR / "index_candidates.jsonl"
HAMRO_JSONL    = OUTPUT_DIR / "profiles_hamropatro.jsonl"
FINAL_CSV      = OUTPUT_DIR / "nepal_election_2082_candidates_education.csv"
FINAL_PARQUET  = OUTPUT_DIR / "nepal_election_2082_candidates_education.parquet"

print("Output directory:", OUTPUT_DIR.resolve())

In [3]:
# Cell 3 – Helper utilities
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

DEVANAGARI_DIGITS = str.maketrans("०१२३४५६७८९", "0123456789")
ZERO_WIDTH = dict.fromkeys(map(ord, ["\u200b", "\u200c", "\u200d", "\ufeff", "\u2060"]), None)

_BAD_EDU_SNIPPETS = [
    "Understanding Nepal's 165 Federal Constituencies",
    "Federal Constituencies",
    "No vote data",
]

# ── Text cleaning ──────────────────────────────────────────────────────────
def _clean_text(s: Optional[str]) -> Optional[str]:
    if s is None:
        return None
    s = str(s).translate(ZERO_WIDTH)
    s = re.sub(r"\s+", " ", s).strip()
    return None if s in ("", "-", "0", "N/A") else s

def _looks_like_education_value(s: str) -> bool:
    """
    Gate-check: returns True if the string plausibly represents an
    academic qualification. Rejects page artefacts and overly long strings.
    """
    t = s.strip()
    if len(t) > 120:   # raised from 80 to allow longer institutional descriptions
        return False
    for bad in _BAD_EDU_SNIPPETS:
        if bad.lower() in t.lower():
            return False

    # Standalone grade number 1–12, with optional pass/suffix
    if re.match(
        r"^(1[0-2]|[1-9])(\s*(pass|पास|उत्तीर्ण|उतिर्ण|उर्तिण|\+\s*2))?$",
        t, re.I
    ):
        return True

    good_tokens = [
        # ── Nepali terms ───────────────────────────────────────────────────
        "साक्षर", "साधारण", "अनौपचारिक",
        "आधारभूत",                          # Grade 6–8 band
        "माध्यमिक", "उच्च माध्यमिक",
        "प्रविणता", "प्रमाणपत्र",            # PCL / proficiency cert variants
        "आई कम",                            # I.Com = Intermediate Commerce
        "स्नातक", "स्‍नातक",
        "स्नातकोत्तर", "स्‍नातकोत्तर", "मास्टर",
        "कक्षा", "एस एल सी", "एसएलसी",
        "एम.फिल", "डाक्टरेट", "पीएचडी",
        "इन्जिनियर", "ईन्जिनेरिङ", "चिकित्सा",
        # ── English qualifications ─────────────────────────────────────────
        "SEE", "SLC", "+2", "10+2", "PCL",
        "diploma", "Diploma",
        "Bachelor", "bachelor", "bachlor", "bacherlor", "BALLB", "ballb",
        "BBS", "BBA", "BSc", "BA", "BE", "LLB",
        "Master", "master", "Masters", "masters",
        "MBS", "MBA", "MSc", "MA", "LLM",
        "MPhil", "mphil", "M.Phil",
        "PhD", "phd", "Doctorate",
        "MBBS", "MD", "CA", "ACCA",
        "pharmacy", "Pharmacy",
        "engineering", "Engineering",
    ]
    t_low = t.lower()
    return any(tok.lower() in t_low for tok in good_tokens)

def safe_int(x: Optional[str]) -> Optional[int]:
    if x is None:
        return None
    s = str(x).translate(DEVANAGARI_DIGITS)
    m = re.search(r"\d+", s)
    return int(m.group(0)) if m else None

# ── BeautifulSoup helpers ──────────────────────────────────────────────────
def soup_from_html(html: str) -> BeautifulSoup:
    soup = BeautifulSoup(html, "lxml")
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()
    return soup

def html_to_lines(html: str) -> List[str]:
    soup = soup_from_html(html)
    text = soup.get_text("\n", strip=True)
    return [ln.strip() for ln in text.splitlines() if ln.strip()]

# ── Label-based field extractors ───────────────────────────────────────────
def first_value_after_label(
    lines: List[str],
    label: str,
    *,
    skip: Iterable[str] = ("Image", "Symbol", "N/A"),
    max_lookahead: int = 8,
) -> Optional[str]:
    skip_set = set(skip)
    for i, ln in enumerate(lines):
        if ln.strip() == label:
            for j in range(i + 1, min(len(lines), i + 1 + max_lookahead)):
                cand = lines[j].strip()
                if not cand or cand in skip_set:
                    continue
                return cand
    return None

def first_value_before_label(
    lines: List[str],
    label: str,
    *,
    skip: Iterable[str] = ("Image", "Symbol", "N/A"),
    max_lookback: int = 8,
) -> Optional[str]:
    skip_set = set(skip)
    for i, ln in enumerate(lines):
        if ln.strip() == label:
            for j in range(i - 1, max(-1, i - 1 - max_lookback), -1):
                cand = lines[j].strip()
                if not cand or cand in skip_set:
                    continue
                return cand
    return None

# ── JSONL I/O ─────────────────────────────────────────────────────────────
def jsonl_append(path: Path, record: Dict) -> None:
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def jsonl_load(path: Path) -> List[Dict]:
    if not path.exists():
        return []
    out = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                out.append(json.loads(line))
    return out

print("Helpers loaded.")

Helpers loaded.


In [4]:
# Cell 4 – HTTP session + cache
def make_session() -> requests.Session:
    s = requests.Session()
    retry = Retry(
        total=6,
        backoff_factor=0.6,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=("GET",),
        raise_on_status=False,
        respect_retry_after_header=True,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=50, pool_maxsize=50)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; civic-data-research/1.0; +https://election.gov.np)",
        "Accept-Language": "en-US,en;q=0.8,ne;q=0.7",
    })
    return s

def jitter_sleep():
    time.sleep(random.uniform(*REQUEST_DELAY_RANGE))

def get_html(session: requests.Session, url: str, timeout: int = 30) -> str:
    jitter_sleep()
    r = session.get(url, timeout=timeout)
    if r.status_code == 404:
        raise FileNotFoundError(f"404: {url}")
    r.raise_for_status()
    return r.text

# Install HTTP cache
if USE_HTTP_CACHE and requests_cache is not None:
    requests_cache.install_cache(
        cache_name=str(CACHE_PATH),
        backend="sqlite",
        expire_after=CACHE_EXPIRE_SECONDS,
        allowable_methods=("GET",),
        stale_if_error=True,
    )
    print("HTTP cache enabled:", CACHE_PATH)
elif USE_HTTP_CACHE and requests_cache is None:
    print("HTTP cache disabled: install 'requests-cache' to enable it")

session = make_session()
print("Session ready.")

HTTP cache enabled: /Users/bikki/Documents/Election_2082/http_cache
Session ready.


In [5]:
# Cell 5 – Candidate ID discovery via HamroPatro range scan
#
# 2082.live sitemap and listing crawl are both unreliable (cap at ~500 IDs).
# Instead we probe HamroPatro directly: IDs are sequential integers starting
# near 339,000. A valid page returns HTTP 200 with a non-empty <h1>; an
# invalid ID returns 404. We scan the window in parallel (12 threads) so
# the whole discovery finishes in ~5–10 minutes.

def _is_valid_hamro_candidate_page(html: str) -> bool:
    """Return True only if the page looks like a real candidate profile."""
    soup = BeautifulSoup(html, "lxml")
    h1 = soup.find("h1")
    return bool(h1 and h1.get_text(strip=True))


def fetch_ids_from_hamropatro_range(
    session: requests.Session,
    *,
    start:   int = HAMRO_ID_RANGE_START,
    end:     int = HAMRO_ID_RANGE_END,
    workers: int = RANGE_SCAN_WORKERS,
) -> set:
    """
    Threaded range scan over [start, end].
    Submits all probes in parallel, collects results as they complete.
    Returns the set of IDs that are valid candidate pages.
    """
    all_cids = list(range(start, end + 1))
    ids: set = set()
    ids_lock = threading.Lock()

    def probe(cid: int) -> tuple:
        url = f"{BASE_HAMRO}/2082/candidate/{cid}"
        try:
            html = get_html(session, url)
            return cid, _is_valid_hamro_candidate_page(html)
        except FileNotFoundError:
            return cid, False
        except Exception:
            return cid, False

    with ThreadPoolExecutor(max_workers=workers) as ex:
        future_to_cid = {ex.submit(probe, cid): cid for cid in all_cids}
        for fut in tqdm(as_completed(future_to_cid), total=len(all_cids),
                        desc="HamroPatro range scan", unit="ID"):
            cid, valid = fut.result()
            if valid:
                with ids_lock:
                    ids.add(cid)

    print(f"Range scan complete: {len(ids):,} valid candidates "
          f"found in {start:,}–{end:,} ({end - start + 1:,} IDs probed)")
    return ids


print("ID discovery function defined.")

ID discovery function defined.


In [6]:
# Cell 6 – Build candidate index using HamroPatro range scan

def build_candidate_index(
    session: requests.Session,
    *,
    resume: bool = True,
) -> pd.DataFrame:
    existing = {
        int(r["candidate_id"]): r
        for r in (jsonl_load(INDEX_JSONL) if (resume and INDEX_JSONL.exists()) else [])
        if "candidate_id" in r
    }
    print(f"Loaded existing index records: {len(existing):,}")

    if existing:
        print(f"Resuming — skipping range scan (delete {INDEX_JSONL.name} to re-scan).")
        all_ids = set(existing.keys())
    else:
        print("Running HamroPatro range scan …")
        all_ids = fetch_ids_from_hamropatro_range(session)

    ids_sorted = sorted(all_ids)
    print(f"Total unique IDs: {len(ids_sorted):,}")

    if len(ids_sorted) < MIN_EXPECTED_CANDIDATES:
        print(f"⚠️  WARNING: Expected ≥{MIN_EXPECTED_CANDIDATES:,} IDs but found only {len(ids_sorted):,}.")
        print("   Check HAMRO_ID_RANGE_END — the real IDs may go higher than 343,500.")

    # ── Write new IDs to JSONL index ──────────────────────────────────────
    new_added = 0
    for cid in tqdm(ids_sorted, desc="Writing index", unit="candidate"):
        if cid in existing:
            continue
        rec = {
            "candidate_id":   cid,
            "url_hamropatro": f"{BASE_HAMRO}/2082/candidate/{cid}",
            "url_2082live":   f"{BASE_2082LIVE}/candidates/cand-{cid}",
            "source_index":   "hamropatro_range_scan",
            "scraped_at_utc": datetime.now(timezone.utc).isoformat(),
        }
        jsonl_append(INDEX_JSONL, rec)
        existing[cid] = rec
        new_added += 1

    print(f"New index rows added: {new_added:,}")
    return pd.DataFrame(list(existing.values())).sort_values("candidate_id").reset_index(drop=True)

index_df = build_candidate_index(session, resume=True)
print(f"\nIndex size: {len(index_df):,} candidates")
index_df.head(3)

Loaded existing index records: 0
Running HamroPatro range scan …


HamroPatro range scan:   0%|          | 0/4501 [00:00<?, ?ID/s]

Range scan complete: 3,487 valid candidates found in 339,000–343,500 (4,501 IDs probed)
Total unique IDs: 3,487


Writing index:   0%|          | 0/3487 [00:00<?, ?candidate/s]

New index rows added: 3,487

Index size: 3,487 candidates


,candidate_id,url_hamropatro,url_2082live,source_index,scraped_at_utc
0,339300,https://election.hamropatro.com/2082/candidate...,https://2082.live/candidates/cand-339300,hamropatro_range_scan,2026-02-18T14:01:02.978051+00:00
1,339301,https://election.hamropatro.com/2082/candidate...,https://2082.live/candidates/cand-339301,hamropatro_range_scan,2026-02-18T14:01:02.979337+00:00
2,339302,https://election.hamropatro.com/2082/candidate...,https://2082.live/candidates/cand-339302,hamropatro_range_scan,2026-02-18T14:01:02.980024+00:00


In [7]:
# Cell 7 – HamroPatro profile parser + bulk scraper

def parse_hamropatro_candidate_page(html: str, candidate_id: int) -> Dict:
    lines = html_to_lines(html)
    soup  = soup_from_html(html)

    h1 = soup.find("h1")
    candidate_name_np = h1.get_text(strip=True) if h1 else None

    age             = safe_int(first_value_before_label(lines, "Age"))
    gender          = first_value_before_label(lines, "Gender")
    district_top    = first_value_before_label(lines, "District")

    father_name         = first_value_after_label(lines, "Father's Name")
    dob                 = first_value_after_label(lines, "Date of Birth")
    citizenship_district= first_value_after_label(lines, "Citizenship District")
    address             = first_value_after_label(lines, "Address")

    election_year   = first_value_after_label(lines, "Election Year")
    constituency_no = safe_int(first_value_after_label(lines, "Constituency"))
    province        = first_value_after_label(lines, "Province")

    qualification   = first_value_after_label(lines, "Qualification")
    institution     = first_value_after_label(lines, "Institution")
    party           = first_value_after_label(lines, "Political Party", skip=("Image", "Symbol", "N/A"))

    return {
        "candidate_id":          int(candidate_id),
        "url_hamropatro_hamro":   f"{BASE_HAMRO}/2082/candidate/{candidate_id}",
        "name_np":               candidate_name_np,
        "party_np":              party,
        "age":                   age,
        "gender":                gender,
        "district_np":           district_top,
        "father_name_np":        father_name,
        "dob_bs_raw":            dob,
        "citizenship_district_np": citizenship_district,
        "address_np":            address,
        "election_year_bs":      election_year,
        "constituency_no_hamro": constituency_no,
        "province_np":           province,
        "qualification_raw":     qualification,
        "institution_raw":       institution,
        "error":                 None,
        "scraped_at_utc_hamro":  datetime.now(timezone.utc).isoformat(),
    }


def scrape_hamropatro_profiles(
    session: requests.Session,
    candidate_ids: List[int],
    *,
    resume: bool = True,
) -> pd.DataFrame:
    from concurrent.futures import ThreadPoolExecutor, as_completed

    existing = {
        int(r["candidate_id"]): r
        for r in (jsonl_load(HAMRO_JSONL) if (resume and HAMRO_JSONL.exists()) else [])
        if "candidate_id" in r
    }
    print(f"Loaded existing profiles: {len(existing):,}")

    todo = [int(cid) for cid in candidate_ids if int(cid) not in existing]
    print(f"Profiles to fetch: {len(todo):,}")

    def worker(cid: int) -> Optional[Dict]:
        url = f"{BASE_HAMRO}/2082/candidate/{cid}"
        try:
            html = get_html(session, url)
        except FileNotFoundError:
            return {"candidate_id": cid, "url_hamropatro_hamro": url,
                    "error": "404",
                    "scraped_at_utc_hamro": datetime.now(timezone.utc).isoformat()}
        except Exception as e:
            return {"candidate_id": cid, "url_hamropatro_hamro": url,
                    "error": f"request_error:{type(e).__name__}",
                    "scraped_at_utc_hamro": datetime.now(timezone.utc).isoformat()}
        try:
            return parse_hamropatro_candidate_page(html, cid)
        except Exception as e:
            return {"candidate_id": cid, "url_hamropatro_hamro": url,
                    "error": f"parse_error:{type(e).__name__}",
                    "scraped_at_utc_hamro": datetime.now(timezone.utc).isoformat()}

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futures = [ex.submit(worker, cid) for cid in todo]
        for fut in tqdm(as_completed(futures), total=len(futures), desc="Scraping profiles"):
            rec = fut.result()
            if not rec:
                continue
            jsonl_append(HAMRO_JSONL, rec)
            existing[int(rec["candidate_id"])] = rec

    df = pd.DataFrame(list(existing.values())).sort_values("candidate_id").reset_index(drop=True)
    return df


candidate_ids = index_df["candidate_id"].astype(int).tolist()
hamro_df = scrape_hamropatro_profiles(session, candidate_ids, resume=True)

error_count = hamro_df["error"].notna().sum() if "error" in hamro_df.columns else 0
print(f"\nHamroPatro rows:   {len(hamro_df):,}")
print(f"Errors / 404s:     {error_count:,}")
hamro_df.head(3)

Loaded existing profiles: 0
Profiles to fetch: 3,487


Scraping profiles:   0%|          | 0/3487 [00:00<?, ?it/s]


HamroPatro rows:   3,487
Errors / 404s:     0


,candidate_id,url_hamropatro_hamro,name_np,party_np,age,gender,district_np,father_name_np,dob_bs_raw,citizenship_district_np,address_np,election_year_bs,constituency_no_hamro,province_np,qualification_raw,institution_raw,error,scraped_at_utc_hamro
0,339300,https://election.hamropatro.com/2082/candidate...,धर्मराज गुरागाई,पिपुल फर्ष्ट पार्टी,43,पुरुष,झापा,नर प्रसाद गुरागाई,2039-06-06,झापा,झापा गौरादह नगरपालिका गौरादह,2082,5,कोशी प्रदेश,10+2,NaN,None,2026-02-18T14:01:03.649831+00:00
1,339301,https://election.hamropatro.com/2082/candidate...,प्यारे लाल राना,स्वतन्त्र,61,पुरुष,कैलाली,मण्डी राना,2021-04-28,कैलाली,कैलाली धनगढी उपमहानगरपालिका पथरी,2082,5,सुदूरपश्चिम प्रदेश,IA,UP Board India,None,2026-02-18T14:01:03.617001+00:00
2,339302,https://election.hamropatro.com/2082/candidate...,विद्या श्रेष्‍ठ (महर्जन),श्रम संस्कृति पार्टी,49,महिला,मकवानपुर,विष्‍णु लाल श्रेष्‍ठ,2033-06-12,काठमाडौं,काठमाडौं काठमाडौँ महानगरपालिका माक गल्‍ली,2082,1,बागमती प्रदेश,स्नातकोतर,त्रिभुवन विश्वविद्यालय,None,2026-02-18T14:01:03.425877+00:00


In [8]:
# Cell 8 – 2082.live education fallback
# For candidates where HamroPatro is missing qualification, we try the
# 2082.live profile page which often has an "Education" field in English.

def parse_2082live_education(html: str) -> Optional[str]:
    soup = soup_from_html(html)

    # Strategy A: definition list
    for dt in soup.find_all("dt"):
        if dt.get_text(" ", strip=True).lower() == "education":
            dd = dt.find_next_sibling("dd")
            if dd:
                val = _clean_text(dd.get_text(" ", strip=True))
                if val and _looks_like_education_value(val):
                    return val

    # Strategy B: table row
    for tr in soup.find_all("tr"):
        cells = [c.get_text(" ", strip=True) for c in tr.find_all(["th", "td"])]
        if len(cells) >= 2 and cells[0].strip().lower() == "education":
            val = _clean_text(cells[1])
            if val and _looks_like_education_value(val):
                return val

    # Strategy C: label node walk-forward
    label_node = soup.find(string=re.compile(r"^\s*Education\s*$", re.I))
    if label_node:
        cur = label_node.parent
        for _ in range(12):
            cur = cur.find_next() if cur else None
            if not cur:
                break
            txt = _clean_text(cur.get_text(" ", strip=True)) if hasattr(cur, "get_text") else None
            if txt and txt.lower() != "education" and _looks_like_education_value(txt):
                return txt

    return None


def fetch_2082live_education_fallback(session: requests.Session, candidate_id: int) -> Optional[str]:
    url = f"{BASE_2082LIVE}/candidates/cand-{candidate_id}"
    try:
        html = get_html(session, url)
        return parse_2082live_education(html)
    except Exception:
        return None


if FALLBACK_FETCH_2082LIVE_PROFILE:
    # Only fetch 2082.live pages for candidates with missing qualification
    missing_edu_mask = hamro_df["qualification_raw"].isna() if "qualification_raw" in hamro_df.columns else pd.Series([True] * len(hamro_df))
    missing_ids = hamro_df.loc[missing_edu_mask, "candidate_id"].astype(int).tolist()
    print(f"Fetching 2082.live education fallback for {len(missing_ids):,} candidates …")

    fallback_map: Dict[int, Optional[str]] = {}
    for cid in tqdm(missing_ids, desc="2082.live fallback"):
        fallback_map[cid] = fetch_2082live_education_fallback(session, cid)

    hamro_df["education_fallback_2082live"] = hamro_df["candidate_id"].map(fallback_map)
    filled = hamro_df["education_fallback_2082live"].notna().sum()
    print(f"Fallback filled {filled:,} additional education values.")
else:
    print("2082.live fallback disabled.")

Fetching 2082.live education fallback for 663 candidates …


2082.live fallback:   0%|          | 0/663 [00:00<?, ?it/s]

Fallback filled 20 additional education values.


In [9]:
# Cell 9 – Merge, normalise, bucket education & export

def normalize_institution(raw: Optional[str]) -> Optional[str]:
    if raw is None:
        return None
    s = str(raw).strip()
    if not s or s in {"0", "-", "N/A"}:
        return None
    s = s.replace("त्रि.वि", "TU").replace("त्रि.वी", "TU").replace("त्रि.वि.", "TU").strip()
    if s.upper() == "TU":  return "Tribhuvan University (TU)"
    if s.upper() == "KU":  return "Kathmandu University (KU)"
    if s.upper() == "PU":  return "Purbanchal University (PU)"
    return s


def education_bucket(raw: Optional[str]) -> Optional[str]:
    """
    Maps a raw qualification string to one of these ordered buckets:
      Doctorate / PhD
      MPhil
      Master
      Bachelor
      Higher Secondary (Grade 11-12 / 10+2)
      Secondary (Grade 10 / SEE/SLC)
      Lower Secondary (Grade 6-8)
      Primary (Grade 1-5)
      Basic Literacy
      Other / Unclear
    """
    if raw is None:
        return None
    s = str(raw).strip().translate(ZERO_WIDTH)
    s = re.sub(r"\s+", " ", s).strip()
    if not s or s in {"0", "-", "N/A"}:
        return None
    s_low = s.lower()

    # ── 1. Doctorate / PhD ─────────────────────────────────────────────────
    if re.search(r"\bphd\b|पीएचडी|पिएचडी|डाक्टरेट", s_low):
        return "Doctorate / PhD"

    # ── 2. MPhil ───────────────────────────────────────────────────────────
    if re.search(r"\bm\.?\s*phil\b|एम\.?फिल", s_low):
        return "MPhil"

    # ── 3. Master ──────────────────────────────────────────────────────────
    # Covers: masters, master degree, master in <field>, M Pharmacy,
    #         MA/MSc/MBA/MBS/LLM, स्नातकोत्तर, मास्टर
    _MASTER_NP = ("स्नातकोत्तर", "स्\u200dनातकोत्तर", "मास्टर")
    _MASTER_RE = re.compile(
        r"\b(masters?|master\s+degree|master\s+in\s+\w+"
        r"|m\.?\s*pharmacy|m\.?\s*a\.?|m\.?\s*sc\.?"
        r"|mba|m\.?\s*ed\.?|m\.?\s*com\.?|mbs|llm)\b"
    )
    if any(p in s for p in _MASTER_NP) or _MASTER_RE.search(s_low):
        return "Master"

    # ── 4. Bachelor ────────────────────────────────────────────────────────
    # Covers: bachelor(s), bacherlor/bachlor (typos), BALLB,
    #         BA/BSc/BBA/BBS/LLB/BE, स्नातक
    _BACH_NP = ("स्नातक", "स्\u200dनातक")
    _BACH_RE = re.compile(
        r"\b(bachelors?|bacherlors?|bachlors?"
        r"|ballb|b\.?\s*a\.?|b\.?\s*sc\.?"
        r"|b\.?\s*ed\.?|b\.?\s*com\.?|bbs|bba|btech"
        r"|b\.?\s*e\.?|llb)\b"
    )
    if any(p in s for p in _BACH_NP) or _BACH_RE.search(s_low):
        return "Bachelor"

    # ── 5. Professional Degree (medical / CA) ──────────────────────────────
    if re.search(r"\b(mbbs|bds|\bca\b|acca|\bmd\b)\b", s_low):
        return "Professional Degree"

    # ── 6. Higher Secondary (Grade 11-12 / 10+2) ───────────────────────────
    # Covers: +2, 10+2, 10 + 2, 12, १२, grade 11, PCL,
    #         all प्रविणता variants, प्रमाणपत्र तह,
    #         diploma programs (post-SLC 3-yr = PCL equivalent),
    #         आई कम (I.Com), उच्च माध्यमिक,
    #         in-progress entries like '१२ उतिर्ण (बि.बि.एस. रनिङ)'
    _HS_NP = (
        "प्रविणता",        # all: प्रविणता प्रमाण पत्र, प्रविणता तह, प्रविणतापत्र, etc.
        "प्रमाणपत्र",      # standalone प्रमाणपत्र तह entries
        "उच्च माध्यमिक",
        "आई कम",           # I.Com = Intermediate Commerce
    )
    _HS_EN = re.compile(
        r"\b(pcl|diploma|10\s*\+\s*2|\+2|i\.?\s*com"
        r"|isc|i\.?\s*sc|intermediate|proficiency)\b"
    )
    # Standalone 12 / १२ with optional pass word;
    # also catches '१२ उतिर्ण (बि.बि.एस. चौथो वर्ष रनिङ)' (Grade 12 completed)
    _GRADE12_RE = re.compile(
        r"^(10\s*\+\s*2|12|११|१२)(\s*(pass|पास|उत्तीर्ण|उतिर्ण|उर्तिण))?",
        re.I
    )
    # Standalone 11: started higher secondary — placed in this band
    _GRADE11_RE = re.compile(r"^11$")
    if (
        any(p in s for p in _HS_NP)
        or _HS_EN.search(s_low)
        or _GRADE12_RE.match(s)
        or _GRADE11_RE.match(s)
    ):
        return "Higher Secondary (Grade 11-12 / 10+2)"

    # ── 7. Secondary (Grade 10 / SEE/SLC) ─────────────────────────────────
    # Covers: SLC, SEE, standalone '10' / '10 pass' / '10 पास',
    #         माध्यमिक तह, माध्यमिक शिक्षा परिषद UP (Indian Grade-10 equivalent)
    _SEC_NP = (
        "एस एल सी", "एस.एल.सी", "एसएलसी",
        "माध्यमिक",   # catches: माध्यमिक तह AND माध्यमिक शिक्षा परिषद
    )
    _SEC_EN = re.compile(r"\b(see|slc|s\.l\.c)\b")
    _GRADE10_RE = re.compile(r"^10(\s*(pass|पास|उत्तीर्ण|उतिर्ण))?$", re.I)
    if (
        any(p in s for p in _SEC_NP)
        or _SEC_EN.search(s_low)
        or _GRADE10_RE.match(s)
    ):
        return "Secondary (Grade 10 / SEE/SLC)"

    # ── 8. कक्षा N (explicit grade-word entries) ────────────────────────────
    if "कक्षा" in s:
        n = safe_int(s)
        if n is None:  return "School (Grade - Unclear)"
        if n <= 5:     return "Primary (Grade 1-5)"
        if n <= 8:     return "Lower Secondary (Grade 6-8)"
        if n <= 10:    return "Secondary (Grade 10 / SEE/SLC)"
        if n <= 12:    return "Higher Secondary (Grade 11-12 / 10+2)"
        return "School (Grade - Unclear)"

    # ── 9. आधारभूत (Grade 8 / basic education level) ─────────────────────
    if "आधारभूत" in s:
        return "Lower Secondary (Grade 6-8)"

    # ── 10. Basic Literacy ────────────────────────────────────────────────
    # साधारण / साधारण लेखपढ = informal reading & writing (~Grade 2 level).
    # Kept separate from Primary: represents non-formal literacy,
    # not structured school enrollment — analytically more meaningful.
    if "साधारण" in s or "साक्षर" in s or re.search(r"\bliterate\b", s_low):
        return "Basic Literacy"

    return "Other / Unclear"


# ── Merge index + hamropatro profiles ─────────────────────────────────────
final_df = index_df.merge(hamro_df, on="candidate_id", how="left", suffixes=("", "_hamro"))

# ── Build education_raw: prefer hamropatro qualification, else 2082.live fallback
final_df["education_raw"] = final_df["qualification_raw"].map(_clean_text)
if "education_fallback_2082live" in final_df.columns:
    mask = final_df["education_raw"].isna()
    final_df.loc[mask, "education_raw"] = final_df.loc[mask, "education_fallback_2082live"].map(_clean_text)

final_df["education_bucket"]  = final_df["education_raw"].apply(education_bucket)
final_df["institution_norm"]  = final_df.get("institution_raw", pd.Series(dtype=str)).apply(normalize_institution)

final_df["dataset_built_at_utc"] = datetime.now(timezone.utc).isoformat()

# ── Column ordering ───────────────────────────────────────────────────────
preferred_cols = [
    "candidate_id", "url_2082live", "url_hamropatro",
    "name_np", "party_np",
    "age", "gender",
    "province_np", "district_np",
    "education_raw", "education_bucket",
    "institution_raw", "institution_norm",
    "qualification_raw",
    "father_name_np", "dob_bs_raw",
    "citizenship_district_np", "address_np",
    "election_year_bs", "constituency_no_hamro",
    "source_index", "dataset_built_at_utc",
    "scraped_at_utc", "scraped_at_utc_hamro",
]
cols = [c for c in preferred_cols if c in final_df.columns] + \
       [c for c in final_df.columns if c not in preferred_cols]
final_df = final_df[cols]

# ── Export ────────────────────────────────────────────────────────────────
final_df.to_csv(FINAL_CSV, index=False, encoding="utf-8-sig")
print("Saved CSV:", FINAL_CSV)

try:
    final_df.to_parquet(FINAL_PARQUET, index=False)
    print("Saved Parquet:", FINAL_PARQUET)
except ImportError:
    print("Skipped Parquet: install pyarrow or fastparquet to enable.")

final_df.head(5)

Saved CSV: /Users/bikki/Documents/Election_2082/nepal_election_2082_candidates_education.csv
Saved Parquet: /Users/bikki/Documents/Election_2082/nepal_election_2082_candidates_education.parquet


,candidate_id,url_2082live,url_hamropatro,name_np,party_np,age,gender,province_np,district_np,education_raw,...,address_np,election_year_bs,constituency_no_hamro,source_index,dataset_built_at_utc,scraped_at_utc,scraped_at_utc_hamro,url_hamropatro_hamro,error,education_fallback_2082live
0,339300,https://2082.live/candidates/cand-339300,https://election.hamropatro.com/2082/candidate...,धर्मराज गुरागाई,पिपुल फर्ष्ट पार्टी,43,पुरुष,कोशी प्रदेश,झापा,10+2,...,झापा गौरादह नगरपालिका गौरादह,2082,5,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.978051+00:00,2026-02-18T14:01:03.649831+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
1,339301,https://2082.live/candidates/cand-339301,https://election.hamropatro.com/2082/candidate...,प्यारे लाल राना,स्वतन्त्र,61,पुरुष,सुदूरपश्चिम प्रदेश,कैलाली,IA,...,कैलाली धनगढी उपमहानगरपालिका पथरी,2082,5,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.979337+00:00,2026-02-18T14:01:03.617001+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
2,339302,https://2082.live/candidates/cand-339302,https://election.hamropatro.com/2082/candidate...,विद्या श्रेष्‍ठ (महर्जन),श्रम संस्कृति पार्टी,49,महिला,बागमती प्रदेश,मकवानपुर,स्नातकोतर,...,काठमाडौं काठमाडौँ महानगरपालिका माक गल्‍ली,2082,1,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.980024+00:00,2026-02-18T14:01:03.425877+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
3,339303,https://2082.live/candidates/cand-339303,https://election.hamropatro.com/2082/candidate...,चन्द्र प्रकाश सुवेदी,स्वतन्त्र,59,पुरुष,मधेश प्रदेश,बारा,स्नातकोत्तर (अर्थ शास्त्र),...,बारा कलैया उपमहानगरपालिका औधापुर,2082,3,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.980360+00:00,2026-02-18T14:01:03.447193+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
4,339304,https://2082.live/candidates/cand-339304,https://election.hamropatro.com/2082/candidate...,इन्द्र माया पौडेल,स्वतन्त्र,58,महिला,बागमती प्रदेश,चितवन,स्नातक तह,...,चितवन भरतपुर महानगरपालिका दिपेन्द्र नगर,2082,3,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.980779+00:00,2026-02-18T14:01:03.668901+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN


In [10]:
# Cell 10 – QA split: clean vs audit

def qa_and_split(df: pd.DataFrame):
    df = df.copy()

    # Clean up education/institution text
    for col in ["education_raw", "institution_raw"]:
        if col in df.columns:
            df[col] = df[col].map(_clean_text)

    # Sanity-filter education values that look like page artefacts
    if "education_raw" in df.columns:
        bad_mask = df["education_raw"].notna() & (~df["education_raw"].map(_looks_like_education_value))
        df.loc[bad_mask, "education_raw"] = None

    # Valid = no error flag
    if "error" in df.columns:
        df["is_valid"] = df["error"].isna()
    else:
        df["is_valid"] = True

    clean = df[df["is_valid"]].drop(columns=["is_valid"])
    audit = df[~df["is_valid"]].drop(columns=["is_valid"])
    return clean, audit


clean_df, audit_df = qa_and_split(final_df)

clean_path = OUTPUT_DIR / "candidates_clean.csv"
audit_path = OUTPUT_DIR / "candidates_audit.csv"

clean_df.to_csv(clean_path, index=False, encoding="utf-8-sig")
audit_df.to_csv(audit_path, index=False, encoding="utf-8-sig")

print(f"Total rows:         {len(final_df):,}")
print(f"Clean candidates:   {len(clean_df):,}")
print(f"Audit rows (errors):{len(audit_df):,}")
print(f"Saved: {clean_path}")
print(f"Saved: {audit_path}")

if len(audit_df) > 100:
    print("⚠️  audit_df unusually large – review candidates_audit.csv")

Total rows:         3,487
Clean candidates:   3,487
Audit rows (errors):0
Saved: /Users/bikki/Documents/Election_2082/candidates_clean.csv
Saved: /Users/bikki/Documents/Election_2082/candidates_audit.csv


In [14]:
# Cell 11 – QA assertions

def hard_assertions(clean_df: pd.DataFrame, audit_df: pd.DataFrame) -> None:
    assert clean_df["candidate_id"].is_unique, "Duplicate candidate_id in clean_df!"

    if "error" in clean_df.columns:
        assert clean_df["error"].isna().all(), "clean_df still contains error rows!"

    if "gender" in clean_df.columns:
        allowed = {"पुरुष", "महिला", "अन्य"}
        bad = set(clean_df["gender"].dropna().unique()) - allowed
        assert not bad, f"Unexpected gender values: {bad}"

    # Constituency = district + constituency_no_hamro (the number resets per district)
    if "district_np" in clean_df.columns and "constituency_no_hamro" in clean_df.columns:
        n_const = (
            clean_df[["district_np", "constituency_no_hamro"]]
            .dropna()
            .drop_duplicates()
            .shape[0]
        )
        if n_const < 100:
            print(f"⚠️  Only {n_const} unique constituencies found – expected 165.")
        else:
            print(f"   Constituencies present: {n_const} (district + number pairs)")

    print("HARD QA passed ✅")

hard_assertions(clean_df, audit_df)

   Constituencies present: 165 (district + number pairs)
HARD QA passed ✅


In [15]:
# Cell 12 – Summary statistics

print(f"Total candidates:  {len(clean_df):,}")
print(f"Unique IDs:        {clean_df['candidate_id'].nunique():,}")

print("\nEducation bucket distribution:")
print(clean_df["education_bucket"].value_counts(dropna=False).to_string())

if "institution_raw" in clean_df.columns:
    pct_missing_inst = clean_df["institution_raw"].isna().mean() * 100
    print(f"\nMissing institution: {pct_missing_inst:.1f}%")

print("\nSample rows where education is 'Other / Unclear':")
clean_df.loc[clean_df["education_bucket"] == "Other / Unclear"].head(10)

Total candidates:  3,487
Unique IDs:        3,487

Education bucket distribution:
education_bucket
Other / Unclear                          1620
Secondary (Grade 10 / SEE/SLC)            645
Bachelor                                  470
Higher Secondary (Grade 11-12 / 10+2)     242
Basic Literacy                            217
Master                                    215
Lower Secondary (Grade 6-8)                35
School (Grade - Unclear)                   14
Doctorate / PhD                            14
MPhil                                       5
Primary (Grade 1-5)                         5
Professional Degree                         5

Missing institution: 0.0%

Sample rows where education is 'Other / Unclear':


,candidate_id,url_2082live,url_hamropatro,name_np,party_np,age,gender,province_np,district_np,education_raw,...,address_np,election_year_bs,constituency_no_hamro,source_index,dataset_built_at_utc,scraped_at_utc,scraped_at_utc_hamro,url_hamropatro_hamro,error,education_fallback_2082live
1,339301,https://2082.live/candidates/cand-339301,https://election.hamropatro.com/2082/candidate...,प्यारे लाल राना,स्वतन्त्र,61,पुरुष,सुदूरपश्चिम प्रदेश,कैलाली,NaN,...,कैलाली धनगढी उपमहानगरपालिका पथरी,2082,5,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.979337+00:00,2026-02-18T14:01:03.617001+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
5,339305,https://2082.live/candidates/cand-339305,https://election.hamropatro.com/2082/candidate...,थम्मन बहादुर बुढा मगर,नेशनल रिपब्लिक नेपाल,39,पुरुष,लुम्बिनी प्रदेश,रूपन्देही,NaN,...,रूपन्देही बुटवल उपमहानगरपालिका,2082,2,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.981715+00:00,2026-02-18T14:01:03.588058+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
8,339308,https://2082.live/candidates/cand-339308,https://election.hamropatro.com/2082/candidate...,कटक बहादुर शाही,नेपाल मजदुर किसान पार्टी,65,पुरुष,सुदूरपश्चिम प्रदेश,कञ्चनपुर,NaN,...,कञ्चनपुर बेलौरी नगरपालिका,2082,1,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.982111+00:00,2026-02-18T14:01:03.779941+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
10,339310,https://2082.live/candidates/cand-339310,https://election.hamropatro.com/2082/candidate...,चूडा प्रसाद ढकाल,स्वतन्त्र,63,पुरुष,गण्डकी प्रदेश,गोरखा,NaN,...,गोरखा सिरानचोक गाउँपालिका झिम्रेक,2082,2,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.982396+00:00,2026-02-18T14:01:03.863761+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
16,339316,https://2082.live/candidates/cand-339316,https://election.hamropatro.com/2082/candidate...,प्रकृति बासी,नेपाल मजदुर किसान पार्टी,38,महिला,कोशी प्रदेश,सुनसरी,NaN,...,भक्तपुर भक्तपुर नगरपालिका जेंला,2082,3,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.982781+00:00,2026-02-18T14:01:04.408891+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
18,339318,https://2082.live/candidates/cand-339318,https://election.hamropatro.com/2082/candidate...,श्‍याम कुमार भट्रराई खत्री,नेपालका लागि नेपाली पार्टी,70,पुरुष,बागमती प्रदेश,काठमाडौं,NaN,...,काभ्रेपलाञ्चोक भुम्लु गाउँपालिका ठाडो कोत्का,2082,4,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.982955+00:00,2026-02-18T14:01:04.476713+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
20,339320,https://2082.live/candidates/cand-339320,https://election.hamropatro.com/2082/candidate...,मीनु गिरी,नेपाल मजदुर किसान पार्टी,64,पुरुष,सुदूरपश्चिम प्रदेश,कञ्चनपुर,NaN,...,कञ्चनपुर दोधारा चाँदनी नगरपालिका स्‍याउली बजार,2082,3,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.983071+00:00,2026-02-18T14:01:04.335783+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
26,339326,https://2082.live/candidates/cand-339326,https://election.hamropatro.com/2082/candidate...,राजेन्द्र बखुन्छे,नेपाल मजदुर किसान पार्टी,54,पुरुष,बागमती प्रदेश,सिन्धुली,NaN,...,भक्तपुर सूर्यविनायक नगरपालिका दर्शनढुङ्गा,2082,1,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.983408+00:00,2026-02-18T14:01:04.865211+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
27,339327,https://2082.live/candidates/cand-339327,https://election.hamropatro.com/2082/candidate...,शान्त वीर लामा,मंगोल नेशनल अर्गनाइजेसन,43,पुरुष,बागमती प्रदेश,सिन्धुली,+2,...,सिन्धुली सुनकोशी गाउँपालिका,2082,2,hamropatro_range_scan,2026-02-18T14:18:18.117218+00:00,2026-02-18T14:01:02.983459+00:00,2026-02-18T14:01:05.016847+00:00,https://election.hamropatro.com/2082/candidate...,None,NaN
28,339328,https://2082.live/candidates/cand-339328,https://election.hamropatro.com/2082/cand

In [16]:
# Cell 13 – Write run metadata (hashes, counts, timestamps)

from hashlib import sha256

def file_sha256(p: Path) -> str:
    h = sha256()
    with p.open("rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

run_meta = {
    "run_time_utc":      datetime.now(timezone.utc).isoformat(),
    "source_hamropatro": BASE_HAMRO,
    "source_2082live":   BASE_2082LIVE,
    "n_index":           int(len(index_df)),
    "n_clean":           int(len(clean_df)),
    "n_audit":           int(len(audit_df)),
    "clean_file":        str(clean_path),
    "audit_file":        str(audit_path),
    "clean_sha256":      file_sha256(clean_path),
    "audit_sha256":      file_sha256(audit_path),
}

meta_path = OUTPUT_DIR / "run_metadata.json"
meta_path.write_text(json.dumps(run_meta, ensure_ascii=False, indent=2), encoding="utf-8")
print("Wrote run metadata:", meta_path.resolve())
print(json.dumps(run_meta, ensure_ascii=False, indent=2))

Wrote run metadata: /Users/bikki/Documents/Election_2082/run_metadata.json
{
  "run_time_utc": "2026-02-18T14:24:18.249940+00:00",
  "source_hamropatro": "https://election.hamropatro.com",
  "source_2082live": "https://2082.live",
  "n_index": 3487,
  "n_clean": 3487,
  "n_audit": 0,
  "clean_file": "/Users/bikki/Documents/Election_2082/candidates_clean.csv",
  "audit_file": "/Users/bikki/Documents/Election_2082/candidates_audit.csv",
  "clean_sha256": "8831a287fbf3f4fd5acf92cdf513c440f668b20e05af6062b09f9da5e7e9a26c",
  "audit_sha256": "36b8822129299c5e08370393ce490488c5b24f8201725c9f7738bfda35397918"
}
